In [ ]:
!pip install cvzone opencv-python mediapipe==0.10.11

### Importing Libraries

In [ ]:
import csv
import pandas as pd
import os
import time
import cv2
import numpy as np
import mediapipe as mp
from cvzone.HandTrackingModule import HandDetector

print("cv2 version:", cv2.__version__)
print("mediapipe version:", mp.__version__)
print("cvzone HandDetector imported successfully")

In [ ]:
classes = ["thanks", "ok", "silent", "rock"]
num_classes = len(classes)

dataset_dir = "SignLanguageDataset"
os.makedirs(dataset_dir, exist_ok=True)

for class_name in classes:
    class_dir = os.path.join(dataset_dir, class_name)
    os.makedirs(class_dir, exist_ok=True)

In [ ]:
cap = cv2.VideoCapture(0)
detector = HandDetector(detectionCon=0.8, maxHands=1)

current_class_index = 0
image_count_per_class = {class_name: len(os.listdir(os.path.join(dataset_dir, class_name))) for class_name in classes}
total_image_count = sum(image_count_per_class.values())
capturing = False

In [ ]:
print("Press 'c' to start/stop capturing images for the current class.")
print("Press 'n' to move to the next class.")
print("Press 'q' to quit.")

In [ ]:
def add_padding_and_resize(hand_img, size=(224, 224), color=(0, 0, 0)):
    h, w, _ = hand_img.shape
    aspect_ratio = w / h

    if aspect_ratio > 1: # width > height
        new_w = size[0]
        new_h = int(new_w / aspect_ratio)
    else:    # height > width
        new_h = size[1]
        new_w = int(new_h * aspect_ratio)

    #resizing and padding
    resized_hand = cv2.resize(hand_img, (new_w, new_h))
    padded_hand = np.full((size[1], size[0], 3), color, dtype=np.uint8)

    # Center the image
    x_offset = (size[0] - new_w) // 2
    y_offset = (size[1] - new_h) // 2
    padded_hand[y_offset:y_offset+new_h, x_offset:x_offset+new_w] = resized_hand

    grayscale_hand = cv2.cvtColor(padded_hand, cv2.COLOR_BGR2GRAY)
    
    return grayscale_hand

In [ ]:
while True:
    success, img = cap.read()
    if not success:
        continue
    
    hands, img = detector.findHands(img)
    current_class = classes[current_class_index]
    
    # Display logic
    cv2.putText(img, f"Class: {current_class}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    # Display the image counters
    cv2.putText(img, f"Images for {current_class}: {image_count_per_class[current_class]}", (10, 70), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
    cv2.putText(img, f"Total Images: {total_image_count}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)

    
    if capturing and hands:
        for hand in hands:
           
            x, y, w, h = hand['bbox']
            hand_img = img[max(0, y-10):min(img.shape[0], y+h+10), 
                           max(0, x-10):min(img.shape[1], x+w+10)]
            processed_hand = add_padding_and_resize(hand_img)

           
            image_count = image_count_per_class[current_class] + 1
            image_path = os.path.join(dataset_dir, current_class, f"{image_count}.jpg")
            cv2.imwrite(image_path, processed_hand)
            
            # update counters
            image_count_per_class[current_class] += 1
            total_image_count += 1
            time.sleep(0.002)

    cv2.imshow("Image", img)


    key = cv2.waitKey(1) & 0xFF
    if key == ord('c'):
        capturing = not capturing
        print(f"{'Started' if capturing else 'Stopped'} capturing images for class '{current_class}'.")
    elif key == ord('n'):
        capturing = False
        current_class_index = (current_class_index + 1) % num_classes
        print(f"Switched to class '{classes[current_class_index]}'.")
    elif key == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()